### Import Modules, Assign Data Path & Read into GDF

In [28]:
#Modules
import osmnx as ox 
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import LineString, MultiLineString
from shapely.geometry import Point, LineString, Polygon
import networkx as nx
import matplotlib.pyplot as plt
from descartes import PolygonPatch
ox.config(log_console=True, use_cache=True)
ox.__version__
#Read Data into DataFrame
vax_df = pd.read_csv('/Users/zachary/Desktop/WINTER22/Geog490/COVID_VaccineClinics/Covid-19_Vaccination_Provider_Locations_in_the_United_States.csv')
# Convert DataFrame to GeoDataFrame
vax_gdf = gpd.GeoDataFrame(vax_df, geometry=gpd.points_from_xy(vax_df['X'], vax_df['Y']))
#Reproject to UTM 17
vax_gdf.crs = 'EPSG:32617'

2022-02-23 14:20:02 Configured OSMnx 1.1.2
2022-02-23 14:20:02 HTTP response caching is on


/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_54547/4099419668.py:16: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  vax_df = pd.read_csv('/Users/zachary/Desktop/WINTER22/Geog490/COVID_VaccineClinics/Covid-19_Vaccination_Provider_Locations_in_the_United_States.csv')


### Limit to Detroit Metro Area (Which we are defining as Wayne, Oakland, and Macomb County)

In [29]:
#limit by state to Michigan
vax_MI = vax_gdf[vax_gdf['State']== 'MI'] 
#Limit By County to Wayne, Oakland, and Macomb
vax_wayne = vax_MI[vax_MI['county']== 'Wayne']
vax_oakland = vax_MI[vax_MI['county']== 'Oakland'] 
vax_macomb = vax_MI[vax_MI['county']== 'Macomb']
#Agglomerate to one GDF of Detroit Metro
vax_detr_metro = vax_wayne 
vax_detr_metro = vax_detr_metro.append(vax_oakland)
vax_detr_metro = vax_detr_metro.append(vax_macomb)

/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_54547/2395504573.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vax_detr_metro = vax_detr_metro.append(vax_oakland)
/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_54547/2395504573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vax_detr_metro = vax_detr_metro.append(vax_macomb)


### Isochrone Loop

In [57]:
def make_iso_polys(G, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = []
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')

        node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({'id': subgraph.nodes()}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index('id')

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get('geometry',  LineString([f,t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union
        
        # try to fill in surrounded areas so shapes will appear solid and blocks without white space inside them
        if infill:
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys.append(new_iso)
    return isochrone_polys

In [68]:
#set Variables

network_type = 'walk'
trip_time = 20 #in minutes
travel_speed = 4.5 #walking speed in km/hour
meters_per_minute = travel_speed * 1000 / 60 #km per hour to m per minute

In [71]:
def isochrone_shape_generator(place, network_type, trip_time, travel_speed, meters_per_minute):
    detroit_polygons = gpd.GeoDataFrame() 
    for i in range(len(vax_detr_metro)): 
        place = vax_detr_metro['Y'].iloc[i],vax_detr_metro['X'].iloc[i]
        network_type = network_type
        trip_time = trip_time
        travel_speed = travel_speed
        meters_per_minute = meters_per_minute
        # download the street network
        G = ox.graph_from_point(place, network_type=network_type)


        # find the centermost node and then project the graph to UTM
        gdf_nodes = ox.graph_to_gdfs(G, edges=False)
        x, y = gdf_nodes['geometry'].unary_union.centroid.xy
        center_node = ox.get_nearest_node(G, (y[0], x[0]))
        G = ox.project_graph(G)


        # add an edge attribute for time in minutes required to traverse each edge
        for u, v, k, data in G.edges(data=True, keys=True):
            data['time'] = data['length'] / meters_per_minute

        # get one color for each isochrone
        #iso_colors = ox.plot.get_colors(n=len(trip_time), cmap='plasma', start=0, return_hex=True)


        # color the nodes according to isochrone then plot the street network
        '''
        node_colors = {}
        for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
            subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')
            for node in subgraph.nodes():
                node_colors[node] = color
        nc = [node_colors[node] if node in node_colors else 'none' for node in G.nodes()]
        ns = [15 if node in node_colors else 0 for node in G.nodes()]
        #fig, ax = ox.plot_graph(G, node_color=nc, node_size=ns, node_alpha=0.8, node_zorder=2,bgcolor='k', edge_linewidth=0.2, edge_color='#999999')
        '''

        # make the isochrone polygons
        
        isochrone_polys = []
        
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')
        node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
        bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
        isochrone_polys.append(bounding_poly)
        

        # plot the network then add isochrones as colored descartes polygon patches
        #fig, ax = ox.plot_graph(G, show=False, close=False, edge_color='#999999', edge_alpha=0.2, node_size=0, bgcolor='k')
        #for polygon, fc in zip(isochrone_polys, iso_colors):
            #patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.6, zorder=-1)
            #ax.add_patch(patch)
        #plt.show()


        isochrone_polys = make_iso_polys(G, edge_buff=25, node_buff=0, infill=True)
        #fig, ax = ox.plot_graph(G, show=False, close=False, edge_color='#999999', edge_alpha=0.2,node_size=0, bgcolor='k')
        #for polygon, fc in zip(isochrone_polys, iso_colors):
            #patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.6, zorder=-1)
            #ax.add_patch(patch)
        #plt.show()
        gdf_iso_polys = gpd.GeoDataFrame(isochrone_polys)
        detroit_polygons = detroit_polygons.append(gdf_iso_polys)
    return detroit_polygons

In [72]:
isochrone_shape_generator(network_type, trip_time, travel_speed, meters_per_minute)

2022-02-23 14:55:34 Created bbox 1000 m from (42.3963769002206, -83.4806078996323): 42.40537010357553,42.387383696865676,-83.46843020434454,-83.49278559492006
2022-02-23 14:55:34 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 14:55:34 Projected GeoDataFrame to epsg:4326
2022-02-23 14:55:34 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 14:55:34 Projected GeoDataFrame to epsg:4326
2022-02-23 14:55:34 Requesting data within polygon from API in 1 request(s)
2022-02-23 14:55:34 Retrieved response from cache file "cache/57002b79dcee13f1efd36a0e9f1512a90a7b4c00.json"
2022-02-23 14:55:34 Got all network data within polygon from API in 1 request(s)
2022-02-23 14:55:34 Creating graph from downloaded OSM data...
2022-02-23 14:55:35 Created graph with 4232 nodes and 9136 edges
2022-02-23 14:55:35 Added length attributes to graph edges
2022-02-23 14:55:35 Identifying all no

/Users/zachary/opt/anaconda3/envs/final_ML_LR/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


2022-02-23 14:55:41 Created nodes GeoDataFrame from graph
2022-02-23 14:55:41 Created nodes GeoDataFrame from graph
2022-02-23 14:55:41 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 14:55:42 Created edges GeoDataFrame from graph
2022-02-23 14:55:42 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 14:55:42 Created graph from node/edge GeoDataFrames
2022-02-23 14:55:42 Projected graph with 469 nodes and 1300 edges


/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_54547/514891206.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  detroit_polygons = detroit_polygons.append(gdf_iso_polys)


2022-02-23 14:55:48 Created bbox 1000 m from (42.3903256997602, -82.9147948003165): 42.39931890311512,42.38133249640527,-82.90261827909424,-82.92697132153876
2022-02-23 14:55:48 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 14:55:48 Projected GeoDataFrame to epsg:4326
2022-02-23 14:55:48 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 14:55:48 Projected GeoDataFrame to epsg:4326
2022-02-23 14:55:48 Requesting data within polygon from API in 1 request(s)
2022-02-23 14:55:48 Retrieved response from cache file "cache/76c1edeab297279db46d8b790aa123a27fd7a24b.json"
2022-02-23 14:55:48 Got all network data within polygon from API in 1 request(s)
2022-02-23 14:55:48 Creating graph from downloaded OSM data...
2022-02-23 14:55:49 Created graph with 2833 nodes and 6666 edges
2022-02-23 14:55:49 Added length attributes to graph edges
2022-02-23 14:55:49 Identifying all nod

/Users/zachary/opt/anaconda3/envs/final_ML_LR/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


2022-02-23 14:55:54 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 14:55:55 Created edges GeoDataFrame from graph
2022-02-23 14:55:55 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 14:55:55 Created graph from node/edge GeoDataFrames
2022-02-23 14:55:55 Projected graph with 522 nodes and 1588 edges


NodeNotFound: Source 2922302695 not in G

In [49]:
detroit_polygons.head()

""


In [73]:
place = vax_detr_metro['Y'].iloc[1],vax_detr_metro['X'].iloc[1]
network_type = network_type
trip_times = trip_times
travel_speed = travel_speed
meters_per_minute = meters_per_minute

# download the street network
G = ox.graph_from_point(place, network_type=network_type)


# find the centermost node and then project the graph to UTM
gdf_nodes = ox.graph_to_gdfs(G, edges=False)
x, y = gdf_nodes['geometry'].unary_union.centroid.xy
center_node = ox.get_nearest_node(G, (y[0], x[0]))
G = ox.project_graph(G)


# add an edge attribute for time in minutes required to traverse each edge
for u, v, k, data in G.edges(data=True, keys=True):
    data['time'] = data['length'] / meters_per_minute

# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0, return_hex=True)


# color the nodes according to isochrone then plot the street network
node_colors = {}
for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')
    for node in subgraph.nodes():
        node_colors[node] = color
nc = [node_colors[node] if node in node_colors else 'none' for node in G.nodes()]
ns = [15 if node in node_colors else 0 for node in G.nodes()]
        #fig, ax = ox.plot_graph(G, node_color=nc, node_size=ns, node_alpha=0.8, node_zorder=2,bgcolor='k', edge_linewidth=0.2, edge_color='#999999')


        # make the isochrone polygons
isochrone_polys = []
for trip_time in sorted(trip_times, reverse=True):
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)


        # plot the network then add isochrones as colored descartes polygon patches
        #fig, ax = ox.plot_graph(G, show=False, close=False, edge_color='#999999', edge_alpha=0.2, node_size=0, bgcolor='k')
        #for polygon, fc in zip(isochrone_polys, iso_colors):
            #patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.6, zorder=-1)
            #ax.add_patch(patch)
        #plt.show()


isochrone_polys = make_iso_polys(G, edge_buff=25, node_buff=0, infill=True)
        #fig, ax = ox.plot_graph(G, show=False, close=False, edge_color='#999999', edge_alpha=0.2,node_size=0, bgcolor='k')
        #for polygon, fc in zip(isochrone_polys, iso_colors):
            #patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.6, zorder=-1)
            #ax.add_patch(patch)
        #plt.show()
gdf_iso_polys = gpd.GeoDataFrame(isochrone_polys)
detroit_polygons = detroit_polygons.append(gdf_iso_polys)

2022-02-23 14:57:30 Created bbox 1000 m from (42.3903256997602, -82.9147948003165): 42.39931890311512,42.38133249640527,-82.90261827909424,-82.92697132153876
2022-02-23 14:57:30 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 14:57:30 Projected GeoDataFrame to epsg:4326
2022-02-23 14:57:30 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 14:57:30 Projected GeoDataFrame to epsg:4326
2022-02-23 14:57:30 Requesting data within polygon from API in 1 request(s)
2022-02-23 14:57:30 Retrieved response from cache file "cache/76c1edeab297279db46d8b790aa123a27fd7a24b.json"
2022-02-23 14:57:30 Got all network data within polygon from API in 1 request(s)
2022-02-23 14:57:30 Creating graph from downloaded OSM data...
2022-02-23 14:57:30 Created graph with 2833 nodes and 6666 edges
2022-02-23 14:57:31 Added length attributes to graph edges
2022-02-23 14:57:31 Identifying all nod

/Users/zachary/opt/anaconda3/envs/final_ML_LR/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


2022-02-23 14:57:35 Created nodes GeoDataFrame from graph
2022-02-23 14:57:35 Created nodes GeoDataFrame from graph
2022-02-23 14:57:35 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 14:57:37 Created edges GeoDataFrame from graph
2022-02-23 14:57:37 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 14:57:37 Created graph from node/edge GeoDataFrames
2022-02-23 14:57:37 Projected graph with 522 nodes and 1588 edges


/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_54547/3961578979.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  detroit_polygons = detroit_polygons.append(gdf_iso_polys)


In [74]:
detroit_polygons

,0
0,"POLYGON ((341502.0415138075 4694202.765720206,..."
0,"POLYGON ((294860.4613026406 4695961.132099903,..."
0,"POLYGON ((341502.0415138075 4694202.765720206,..."
